<center><img src=img/MScAI_brand.png width=70%></center>

# `eval` and `exec`

If we have a string `s = "5"` and we type `i = int(s)`, it converts `s` to `int`.


If we have a string `s = "5.0"` and we type `x = float(s)`, it converts `s` to `float`.

But if we're **at the command line** (IPython or Jupyter notebook) and we just type `i = 5`, "it just works" -- `i` gets the right type without any special conversion step.

Since Python knows how to do this, we might ask: why can't we get Python to convert *any* string
to **whatever value and type it would have been**, if we had just typed that string as input on the command-line?

That's what `eval` does. If `s = "5"`, then `eval(s)` is the same as `int(s)`. If `s = "5.0"`, then `eval(s)` is the same as `float(s)`. If `s` is a complex expression then `eval(s)` will evaluate `s` just as if it had been typed at the command-line.

<center><img src=img/repl.svg width=30%></center>

In fact, a command-line (not only in Python) is often called a *read-eval-print* loop for this reason.

### An `eval` application

Remember we mentioned when studying Scikit-Learn that when we create a logistic regression `lr = LogisticRegression()`, that object has a string representation like this:

```python
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
```


That string has a nice property: we could copy and paste it into IPython or Jupyter Notebook to re-create the object: `lr = <copy and paste here!>`


That doesn't seem much more useful than just typing `lr = LogisticRegression()`. 

But if we had set many parameters with non-default values it could be very useful:

`lr = LogisticRegression(C=10.0, solver='lbfgs', max_iter=1000)`

Then just typing `lr` will give us a nice, simple textual representation of the model we created. We could save it and later copy-and-paste it into IPython to recreate the model with the same hyperparameters.

However, what if we wanted to do this for many models in a loop with different hyperparameters? In other words we have created many models and saved their text representations, and later want to do something else with them. We can't just copy-and-paste them in a loop.

But that's where `eval` comes in:

```python
for fname in fnames:
    s = open(fname).read()
    lr = eval(s)
```

### Python ~ JSON

`eval` works well with all sorts of Python *literals*, e.g. `int`s, `float`s, `str`s, `{}` for dictionaries, `[]` for lists, and so on. Therefore, we might save a Python object to disk just by taking the string representation, and then load it back in later, effectively treating Python as if it was JSON. This can be very useful in **logging** from a long-running program.

In [9]:
x = {"a": [3, 4, 5], "b": [5, 6, 7]}
open("data/eval_test.txt", "w").write(repr(x))
y = eval(open("data/eval_test.txt").read())
y

{'a': [3, 4, 5], 'b': [5, 6, 7]}

(There is also the `json` module in the standard library if we want to deal with JSON properly.)

### `__str__` and `__repr__`

Notice that above we used `repr(x)`, not `str(x)`.

* `__str__` is supposed to give a descriptive string for human consumption;
* `__repr__` is supposed to give a string which can usually be `eval`-ed to recreate the original object.

For many classes, including `dict`, these are just the same thing. If an object `c` has `__repr__` but not `__str__`, then `str(c)` will default to calling `c.__repr__()`. 

### Security with `eval`

`eval` is dangerous, because it executes a piece of code. If you write a program with `eval` and provide the input, e.g. from a file you saved earlier, that's fine. If you write a program with `eval` and accept input from anonymous users over the internet, you might have some problems.



### `exec`

As we have seen, `eval` allows us to evaluate a Python **expression**. 

An expression in Python that has a single value, like a generalisation of a **formula** in maths. It can have arithmetic, Boolean operators, function calls, object constructors, and so on. It does not have assignment statements, conditionals, loops, function definitions (except maybe `lambda`).

But sometimes, we might have Python code `s` which is not a single expression. In such a case, `eval` doesn't work. It expects a single expression only:

In [3]:
s = """
def f(x):
    if x > 3:
        print("the result is excessive")
for i in range(5):
    f(i)
"""
c = eval(s)


SyntaxError: invalid syntax (<string>, line 2)

There is another function, `exec`, which **executes** arbitrary Python code.

In [4]:
exec(s)

the result is excessive


An arbitrary piece of code doesn't have a value, in contrast to an expression. So, `exec` doesn't return a value. (There is a simple hack to access values calculated inside `exec`, but we won't go any further with this.)

Everything we said about `eval` security applies even more so to `exec`, because it has even greater flexibility!

### Exercise

* Can you think of a Python object which is *not* correctly and fully re-created if we write out its `repr` and then read it in and `eval` the result?

### Solution

* E.g. in Scikit-Learn we can recreate a `LogisticRegression` in this way, but if we have `fit()`ted then the internal parameters won't be part of the `repr`. Use `pickle` or similar instead.